In [ ]:
import os
import logging

import psycopg2
import community
import numpy as np
import pandas as pd
import networkx as nx

import matplotlib as mp
%matplotlib inline
import matplotlib.pyplot as plt

from IPython.display import display

from tqdm.notebook import tqdm

from nltk.corpus import stopwords as sw

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

import utils as ut

In [ ]:
fmt = '%(asctime)s : %(levelname)s : %(message)s'
logging.basicConfig(format=fmt, level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None

In [ ]:
os.chdir(os.path.expanduser('~/github/masthesis/'))

In [ ]:
os.makedirs('data/twitter/community/', exist_ok=True)

# Load data

### User profile data

In [ ]:
user_data = pd.read_csv('data/samples/twitter/user-data.csv', index_col='user_id')
display(user_data.shape)

In [ ]:
user_show = pd.read_csv('data/samples/twitter/user-show.csv')
display(user_show.shape)

### Follow graph

In [ ]:
follow_graph = pd.read_csv('data/samples/twitter/follow-graph.csv')
follow_graph = follow_graph.rename({'source_user_id': 'source', 'target_user_id': 'target'}, axis=1)

fg = nx.from_pandas_edgelist(follow_graph, source='source', target='target',
                             edge_attr=None, create_using=nx.DiGraph())

display(follow_graph.shape)

In [ ]:
follow_graph.head()

In [ ]:
fg.order()

In [ ]:
fg.size()

# Confirm connectivity

In [ ]:
ucomps = [x for x in nx.connected_components(fg.to_undirected())]
display(len(ucomps))

In [ ]:
scomps = [x for x in nx.strongly_connected_components(fg)]
display(len(scomps))

In [ ]:
pd.Series([len(x) for x in scomps]).value_counts()

# Community detection + centrality

## Graph as-is

In [ ]:
partition = community.best_partition(fg.to_undirected())
partition = pd.Series(partition, name='community')

comms = pd.DataFrame(partition).merge(user_data[['screen_name', 'name', 'description', 'location', 'is_radio']],
                                      how='inner', left_index=True, right_index=True)
comms.index.name = 'user_id'

In [ ]:
comms.community.value_counts()

In [ ]:
comms.loc[comms.is_radio, 'community'].value_counts()

In [ ]:
comms.sample(frac=1).groupby('community').head(10).sort_values('community')

## Quotient graph collapsing shows

The `nx.quotient_graph()` function is very, very slow...

In [ ]:
qg = fg.to_undirected().copy()

In [ ]:
for sh in user_show.show_id.unique():
    nodes = {i for i in user_show.loc[user_show.show_id == sh, 'user_id']}
    
    rep = list(nodes)[0]
    for n in nodes - {rep}:
        if n not in qg.nodes: # has no followers
            continue
        
        edges = list(qg.edges(n))
        for e1, e2 in edges:
            assert e1 == n

            if e2 == rep:
                continue
            else:
                qg.add_edge(rep, e2)
        
        qg.remove_node(n)

In [ ]:
qpartition = community.best_partition(qg)
qpartition = pd.Series(qpartition, name='community')

qcomms = pd.DataFrame(qpartition).merge(user_data[['screen_name', 'name', 'description', 'location', 'is_radio']],
                                        how='inner', left_index=True, right_index=True)
qcomms.index.name = 'user_id'

In [ ]:
qcomms.community.value_counts()

In [ ]:
qcomms.loc[qcomms.is_radio, 'community'].value_counts()

In [ ]:
qcomms.sample(frac=1).groupby('community').head(10).sort_values('community')

## Betweenness centrality

In [ ]:
bc = pd.Series(nx.betweenness_centrality(fg), name='centrality')
bc = pd.DataFrame(bc).reset_index().rename({'index': 'user_id'}, axis=1).set_index('user_id')

In [ ]:
qbc = pd.Series(nx.betweenness_centrality(qg), name='centrality')
qbc = pd.DataFrame(qbc).reset_index().rename({'index': 'user_id'}, axis=1).set_index('user_id')

# Match to shows

## Graph as-is

In [ ]:
rcomms = comms.merge(user_show, how='inner', right_on='user_id', left_index=True)

In [ ]:
rcomms.community.value_counts()

In [ ]:
dupes = rcomms.groupby('show_id').community.nunique().sort_values()
dupes = dupes[dupes > 1].index.tolist()

rcomms.loc[rcomms.show_id.isin(dupes), ['show_id', 'show_name']].drop_duplicates()

In [ ]:
show_community = rcomms.loc[~rcomms.show_id.isin(dupes), ['show_id', 'community']]
show_community = show_community.reset_index().drop('index', axis=1).set_index('show_id').community

In [ ]:
show_community.value_counts()

## Quotient graph

In [ ]:
rqcomms = qcomms.merge(user_show, how='inner', right_on='user_id', left_index=True)

In [ ]:
rqcomms.community.value_counts()

In [ ]:
dupes = rqcomms.groupby('show_id').community.nunique().sort_values()
dupes = dupes[dupes > 1].index.tolist()

rcomms.loc[rcomms.show_id.isin(dupes), ['show_id', 'show_name']].drop_duplicates()

In [ ]:
qshow_community = rqcomms.loc[~rqcomms.show_id.isin(dupes), ['show_id', 'community']]
qshow_community = qshow_community.reset_index().drop('index', axis=1).set_index('show_id').community

In [ ]:
qshow_community.value_counts()

## Betweenness centrality

In [ ]:
bc_show = qbc.merge(user_show, how='inner', right_on='user_id', left_index=True)
bc_show = bc_show[['show_id', 'centrality']].set_index('show_id')

bc_show.head()

In [ ]:
bc_show.centrality.hist(bins=50)

# Write out data

In [ ]:
nx.write_edgelist(qg, path='data/twitter/community/quotient-follow-graph.csv', delimiter=',')

In [ ]:
comms[['community']].to_csv('data/twitter/community/follow-community-all.csv')

In [ ]:
qcomms[['community']].to_csv('data/twitter/community/quotient-follow-community-all.csv')

In [ ]:
bc.to_csv('data/twitter/community/follow-centrality.csv')

In [ ]:
qbc.to_csv('data/twitter/community/quotient-follow-centrality.csv')

In [ ]:
show_community.to_csv('data/twitter/community/follow-community-by-show.csv')

In [ ]:
qshow_community.to_csv('data/twitter/community/quotient-follow-community-by-show.csv')

In [ ]:
bc_show.to_csv('data/twitter/community/quotient-follow-centrality-by-show.csv')